# 03 - Querying with prompting

In [ ]:
%reload_ext dotenv
%dotenv

import os

from graphrag_toolkit.lexical_graph.storage.graph.neo4j_graph_store_factory import Neo4jGraphStoreFactory
from graphrag_toolkit.lexical_graph.storage import GraphStoreFactory
from graphrag_toolkit.lexical_graph.storage import VectorStoreFactory
from graphrag_toolkit.lexical_graph import set_logging_config

set_logging_config('INFO')

# Register the Neo4j backend with the factory
GraphStoreFactory.register(Neo4jGraphStoreFactory)

# Create graph and vector stores
graph_store = GraphStoreFactory.for_graph_store(os.environ['GRAPH_STORE'])
vector_store = VectorStoreFactory.for_vector_store(os.environ['VECTOR_STORE'])

### TraversalBasedRetriever

See [TraversalBasedRetriever](https://github.com/awslabs/graphrag-toolkit/blob/main/docs/lexical-graph/querying.md#traversalbasedretriever).

In [ ]:
from graphrag_toolkit.lexical_graph import LexicalGraphQueryEngine

query_engine = LexicalGraphQueryEngine.for_traversal_based_search(
    graph_store, 
    vector_store,
    streaming=True
)

response = query_engine.query("What are the similarities and differences between Neptune Database and Neptune Analytics?")

print(response.print_response_stream())

## Set prompt from disk

In [ ]:
from graphrag_toolkit.lexical_graph import LexicalGraphQueryEngine
from graphrag_toolkit.lexical_graph.prompts.file_prompt_provider import FilePromptProvider
from graphrag_toolkit.lexical_graph.prompts.prompt_provider_config import FilePromptProviderConfig

# Setup prompt provider
prompt_provider = FilePromptProvider(
    FilePromptProviderConfig(
        system_prompt_file="prompts/system_prompt.txt",
        user_prompt_file="prompts/user_prompt.txt"
    )
)

# Create query engine with prompt provider
query_engine = LexicalGraphQueryEngine.for_traversal_based_search(
    graph_store, 
    vector_store,
    streaming=True,
    prompt_provider=prompt_provider
)

response = query_engine.query("What are the similarities and differences between Neptune Database and Neptune Analytics?")

print(response.print_response_stream())


## Set prompt from S3

In [ ]:
from graphrag_toolkit.lexical_graph import LexicalGraphQueryEngine
from graphrag_toolkit.lexical_graph.prompts.s3_prompt_provider import S3PromptProvider
from graphrag_toolkit.lexical_graph.prompts.prompt_provider_config import S3PromptProviderConfig

# Setup S3 prompt provider
prompt_provider = S3PromptProvider(
    S3PromptProviderConfig(
        bucket="ccms-prompts",
        prefix="prompts",
        aws_region="ap-south-1",
        aws_profile="padmin",
        system_prompt_file="system_prompt.txt",
        user_prompt_file="user_prompt.txt"
    )
)

# Create query engine with S3 prompts
query_engine = LexicalGraphQueryEngine.for_traversal_based_search(
    graph_store, 
    vector_store,
    streaming=True,
    prompt_provider=prompt_provider
)

response = query_engine.query("What are the similarities and differences between Neptune Database and Neptune Analytics?")

print(response.print_response_stream())


## Bedrock Manage Prompt

In [ ]:
from graphrag_toolkit.lexical_graph import LexicalGraphQueryEngine
from graphrag_toolkit.lexical_graph.prompts.prompt_provider_config import BedrockPromptProviderConfig

# Setup Bedrock prompt provider
prompt_provider = BedrockPromptProviderConfig(
    aws_region="us-east-1",
    aws_profile="padmin",
    system_prompt_arn="KEOXPXUM00",
    user_prompt_arn="TSF4PI4A6C",
    system_prompt_version="1",
    user_prompt_version="1"
).build()

# Create query engine with Bedrock prompts
query_engine = LexicalGraphQueryEngine.for_traversal_based_search(
    graph_store, 
    vector_store,
    streaming=True,
    prompt_provider=prompt_provider
)

response = query_engine.query("What are the similarities and differences between Neptune Database and Neptune Analytics?")

print(response.print_response_stream())
